In [7]:
import requests
import pandas as pd
import time

POLYGON_KEY = "ML8KNIhH8hbBS9Cv_w9YcHfwqEpp3IQZ"  # 🔐 Replace with your actual API key

# Step 1: Load all unique tickers from stocks.csv
df = pd.read_csv("stocks.csv", parse_dates=["date"])
symbols = sorted(df["symbol"].dropna().unique())

# Step 2: Query Polygon's ticker details
def get_market_cap(symbol):
    url = f"https://api.polygon.io/v3/reference/tickers/{symbol.upper()}"
    params = {"apiKey": POLYGON_KEY}
    try:
        res = requests.get(url, params=params)
        res.raise_for_status()
        data = res.json()
        market_cap = data.get("results", {}).get("market_cap", None)
        name = data.get("results", {}).get("name", "")
        return {"symbol": symbol, "market_cap": market_cap, "company_name": name}
    except Exception as e:
        print(f"⚠️ {symbol}: {e}")
        return {"symbol": symbol, "market_cap": None, "company_name": None}

# Step 3: Loop with rate-limiting (optional for safety)
results = []
for i, sym in enumerate(symbols, 1):
    print(f"📡 [{i}/{len(symbols)}] {sym}")
    info = get_market_cap(sym)
    results.append(info)
    time.sleep(0.5)  # ✅ adjust if needed — less aggressive with paid plan

# Step 4: Save to CSV
market_cap_df = pd.DataFrame(results)
market_cap_df = market_cap_df.dropna(subset=["market_cap"])
market_cap_df.to_csv("market_cap.csv", index=False)

print("✅ Done. Saved to market_cap.csv")


📡 [1/181] AAPL
📡 [2/181] ABNB
📡 [3/181] ACN
📡 [4/181] ADBE
📡 [5/181] ADI
📡 [6/181] ADM
📡 [7/181] ADSK
📡 [8/181] AKAM
📡 [9/181] AMAT
📡 [10/181] AMD
📡 [11/181] AMZN
📡 [12/181] ANET
📡 [13/181] APH
📡 [14/181] APTV
📡 [15/181] AVGO
📡 [16/181] AZO
📡 [17/181] BBY
📡 [18/181] BF.B
📡 [19/181] BG
📡 [20/181] BKNG
📡 [21/181] CAG
📡 [22/181] CCL
📡 [23/181] CDNS
📡 [24/181] CDW
📡 [25/181] CHD
📡 [26/181] CHTR
📡 [27/181] CL
📡 [28/181] CLX
📡 [29/181] CMCSA
📡 [30/181] CMG
📡 [31/181] COST
📡 [32/181] CPB
📡 [33/181] CRM
📡 [34/181] CRWD
📡 [35/181] CSCO
📡 [36/181] CTSH
📡 [37/181] CZR
📡 [38/181] DASH
📡 [39/181] DDOG
📡 [40/181] DECK
📡 [41/181] DELL
📡 [42/181] DG
📡 [43/181] DHI
📡 [44/181] DIS
📡 [45/181] DLTR
📡 [46/181] DPZ
📡 [47/181] DRI
📡 [48/181] EA
📡 [49/181] EBAY
📡 [50/181] EL
📡 [51/181] ENPH
📡 [52/181] EPAM
📡 [53/181] EXPE
📡 [54/181] F
📡 [55/181] FFIV
📡 [56/181] FICO
📡 [57/181] FOX
📡 [58/181] FOXA
📡 [59/181] FSLR
📡 [60/181] FTNT
📡 [61/181] GDDY
📡 [62/181] GEN
📡 [63/181] GIS
📡 [64/181] GLW
📡 [65/181] GM
📡 [66/1

In [8]:
import pandas as pd

# Load your raw market cap data
df = pd.read_csv("market_cap.csv")

# Step 1: Parse market cap strings like '3.2T', '750B', '44M'
def parse_market_cap(value):
    if pd.isna(value):
        return None
    value = str(value).upper().replace("$", "").strip()
    try:
        if value.endswith("T"):
            return float(value[:-1]) * 1e12
        elif value.endswith("B"):
            return float(value[:-1]) * 1e9
        elif value.endswith("M"):
            return float(value[:-1]) * 1e6
        else:
            return float(value)
    except:
        return None

df["cap_numeric"] = df["market_cap"].apply(parse_market_cap)

# Step 2: Assign cap score (1 = mega, 5 = small)
def get_cap_score(cap):
    if pd.isna(cap):
        return 5  # treat unknowns as smallest
    elif cap >= 200e9:
        return 1
    elif cap >= 50e9:
        return 2
    elif cap >= 10e9:
        return 3
    elif cap >= 2e9:
        return 4
    else:
        return 5

df["cap_score"] = df["cap_numeric"].apply(get_cap_score)

# Step 3: Optional emoji mapping (for Streamlit)
emoji_map = {1: "🏆", 2: "🥈", 3: "🥉", 4: "🟢", 5: "🟡"}
df["cap_emoji"] = df["cap_score"].map(emoji_map)

# Step 4: Save cleaned version
output_cols = ["symbol", "company_name", "market_cap", "cap_numeric", "cap_score", "cap_emoji"]
df[output_cols].to_csv("market_cap.csv", index=False)

print("✅ Updated market_cap.csv with cap_score and cap_emoji columns.")


✅ Updated market_cap.csv with cap_score and cap_emoji columns.
